In [ ]:
import pandas as pd
import numpy as np

In [ ]:
student_data = [[3, 620, 688, 638, 688], [5, 534, 602, 606, 700], [9, 487, 611, 477, 578]]
exam_score_df = pd.DataFrame(student_data, columns = ['student_ID', 'LANG', 'MAT', 'HIST', 'SCI'])
exam_score_df.set_index('student_ID')

program_data = [[101, 20, 30, 25, 25, 'N'], [102, 40, 10, 50, 50, 'Y']]
school_weight_df = pd.DataFrame(program_data, columns = ['program_code', '%LANG','%MAT','%HIST','%SCI', 'Alternative_Score'])
school_weight_df.set_index('program_code')

In [ ]:
exam_score_df

In [ ]:
school_weight_df

In [ ]:
def calc_score(student_ID, program_code):
    '''
    For a given student and program, returns students score for that program.
    '''
    if school_weight_df.loc[program_code]['Alternative_Score'] == 'N':

        return np.dot(np.array(exam_score_df.loc[student_ID][['LANG', 'MAT', 'HIST', 'SCI']]), 
                      np.array(school_weight_df.loc[program_code][['%LANG','%MAT','%HIST','%SCI']]))

    elif school_weight_df.loc[program_code]['Alternative_Score'] == 'Y':

        history_score = np.dot(np.array(exam_score_df.loc[student_ID][['LANG', 'MAT', 'HIST']]), 
                               np.array(school_weight_df.loc[program_code][['%LANG','%MAT','%HIST']]))

        science_score = np.dot(np.array(exam_score_df.loc[student_ID][['LANG', 'MAT', 'SCI']]), 
                               np.array(school_weight_df.loc[program_code][['%LANG','%MAT','%SCI']]))

        return max(history_score, science_score)

In [7]:
scores = np.einsum('ij,kj->kij',exam_scores, school_weights) #(1500,5000,4)

#Alternative == N
n_alt = scores[~alt_flag].sum(-1)   #(737, 5000)

#Alternative == Y
lm = scores[alt_flag,:,:2].sum(-1)  #(763, 5000, 2) -> (763, 5000)
h = scores[alt_flag,:,2]            #(763, 5000)
s = scores[alt_flag,:,3]            #(763, 5000)
y_alt = np.maximum(lm+h, lm+s)      ##(763, 5000)

In [18]:
i = 100
exam_scores[i:i+50].shape

(50, 4)

In [40]:
import numpy as np
import pandas as pd
from tqdm import notebook

exam_scores = np.random.randint(300,800,(100000,4))
school_weights = np.random.randint(10,50,(1500,4))
alt_flag = np.random.randint(0,2,(1500,), dtype=bool) #0 for N, 1 for Y
batch = 5000

n_alts = []
y_alts = []

for i in notebook.tqdm(range(0,100000,batch)):
    scores = np.einsum('ij,kj->kij', exam_scores[i:i+batch], school_weights) #(1500,5000,4)

    #Alternative == N
    n_alt = scores[~alt_flag].sum(-1).T   #(5000, 766)

    #Alternative == Y
    lm = scores[alt_flag,:,:2].sum(-1)    #(734, 5000, 2) -> (734, 5000)
    h = scores[alt_flag,:,2]              #(734, 5000)
    s = scores[alt_flag,:,3]              #(734, 5000)
    y_alt = np.maximum(lm+h, lm+s).T      #(5000, 734)
    
    n_alts.append(n_alt)
    y_alts.append(y_alt)

final_n_alts = np.vstack(n_alts)
final_y_alts = np.vstack(y_alts)

print(final_n_alts.shape)
print(final_y_alts.shape)


(100000, 778)
(100000, 722)


In [49]:
# My 3D raster containing values; shape = (time, x, y)
values = np.random.rand(10, 55, 60)

# A 2D raster containing start indices for the averaging
start_index = np.random.randint(0, 4, size=(values.shape[1], values.shape[2]))

# A 2D raster containing end indices for the averaging
end_index = np.random.randint(5, 9, size=(values.shape[1], values.shape[2]))

# Initialise an array that will contain results
mean_array = np.zeros_like(values[0, :, :])

# Loop over 3D raster to calculate the average between indices on axis 0
for i in range(0, values.shape[1]):
    for j in range(0, values.shape[2]):
        mean_array[i, j] = np.mean(values[start_index[i, j]: end_index[i, j], i, j], axis=0)

In [113]:
mean_array.shape

(55, 60)

In [ ]:
np.mean(values[start_index[0,0]: end_index[0,0], i, j], axis=0)

In [94]:
values[start_index[0,0]: end_index[0,0], 0, 0]

array([0.04578405, 0.68914886, 0.15233235, 0.92129213])

In [90]:
values.shape

(10, 55, 60)

In [95]:
start_index.shape

(55, 60)

In [96]:
end_index.shape

(55, 60)

In [107]:
slices = np.vectorize(slice)(start_index.flatten(), end_index.flatten())


slice(3, 7, None)

In [118]:
values[np.r_[slices[0]],0,0].mean(0)

0.45213934756553775

In [134]:
np.r_[slices[1]]

array([1, 2, 3, 4, 5, 6])

In [138]:
np.r_[slices].reshape(55,60)

(55, 60)

In [ ]:
np.take()

In [149]:
end_index-start_index

array([[4, 6, 7, ..., 4, 5, 7],
       [7, 5, 6, ..., 2, 5, 7],
       [6, 4, 6, ..., 7, 2, 4],
       ...,
       [5, 6, 5, ..., 4, 4, 5],
       [5, 5, 7, ..., 4, 8, 8],
       [6, 3, 7, ..., 4, 7, 6]])

In [161]:
np.r_[slices[0]]

array([3, 4, 5, 6])

In [162]:
np.r_[slices[1]]

array([1, 2, 3, 4, 5, 6])

In [163]:
values[np.ix_([[3,4,5,6],[1,2,3,4,5,6]])].mean(0)

/Users/akshay/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


IndexError: arrays used as indices must be of integer (or boolean) type

In [144]:
start_index.shape

(55, 60)

In [139]:
i = np.arange(values.shape[0])[:, None, None]
mean_array_2 = np.where((i >= start_index) & (i < end_index), values, 0).sum(0) / (end_index - start_index)
np.allclose(mean_array, mean_array_2)

True

In [133]:
values.flatten()[np.r_[slices[1]]]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

array([slice(3, 7, None), slice(1, 7, None), slice(0, 7, None), ...,
       slice(3, 7, None), slice(0, 7, None), slice(0, 6, None)],
      dtype=object)

In [131]:
values[np.r_[slices[1]],0,1].mean(0)

0.5060096517897337

In [ ]:
np.mean()

In [ ]:
values

In [83]:
values[:,3:7]

array([[[0.90585974, 0.67437564, 0.44775102, ..., 0.29507425,
         0.97570608, 0.68645807],
        [0.55678876, 0.26923511, 0.97362446, ..., 0.26959906,
         0.78424757, 0.64538441],
        [0.75226496, 0.43052415, 0.00939038, ..., 0.02186048,
         0.09784362, 0.16870332],
        [0.33922807, 0.55223871, 0.17701378, ..., 0.08802453,
         0.53673236, 0.08066298]],

       [[0.98685788, 0.61374661, 0.06998783, ..., 0.95155097,
         0.83762397, 0.49351058],
        [0.49919388, 0.47033415, 0.8408022 , ..., 0.57482913,
         0.5549347 , 0.38827401],
        [0.73666848, 0.04183017, 0.05085872, ..., 0.44363453,
         0.05956699, 0.66909371],
        [0.53980613, 0.87422528, 0.52551556, ..., 0.35795426,
         0.97810507, 0.19949406]],

       [[0.74396403, 0.70587141, 0.80963927, ..., 0.28528235,
         0.90056066, 0.65865189],
        [0.68903568, 0.05299841, 0.13723521, ..., 0.08179182,
         0.96324639, 0.22137117],
        [0.93207917, 0.69900467, 0.9

In [51]:
values.shape

(10, 55, 60)

In [53]:
start_index.shape

(55, 60)

In [54]:
end_index.shape

(55, 60)

In [58]:
start_index[0]

array([3, 1, 0, 2, 3, 1, 0, 1, 0, 3, 0, 3, 3, 1, 0, 2, 0, 3, 3, 0, 3, 0,
       1, 2, 1, 2, 1, 1, 2, 3, 0, 0, 0, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3,
       0, 1, 1, 0, 3, 1, 3, 2, 3, 1, 0, 0, 1, 2, 2, 1])

In [59]:
end_index[0]

array([7, 7, 7, 7, 7, 8, 6, 5, 7, 8, 8, 7, 7, 5, 5, 6, 7, 8, 8, 5, 8, 8,
       5, 7, 6, 8, 8, 5, 6, 7, 5, 5, 5, 5, 7, 8, 6, 5, 7, 6, 6, 6, 6, 7,
       8, 7, 6, 8, 8, 5, 6, 8, 5, 6, 6, 7, 6, 6, 7, 8])

In [66]:
np.stack([start_index, end_index])

array([[[3, 1, 0, ..., 2, 2, 1],
        [0, 2, 1, ..., 3, 0, 1],
        [1, 2, 1, ..., 0, 3, 1],
        ...,
        [1, 2, 3, ..., 1, 1, 1],
        [2, 0, 1, ..., 3, 0, 0],
        [1, 3, 1, ..., 3, 0, 0]],

       [[7, 7, 7, ..., 6, 7, 8],
        [7, 7, 7, ..., 5, 5, 8],
        [7, 6, 7, ..., 7, 5, 5],
        ...,
        [6, 8, 8, ..., 5, 5, 6],
        [7, 5, 8, ..., 7, 8, 8],
        [7, 6, 8, ..., 7, 7, 6]]])

In [62]:
start_index

array([[3, 1, 0, ..., 2, 2, 1],
       [0, 2, 1, ..., 3, 0, 1],
       [1, 2, 1, ..., 0, 3, 1],
       ...,
       [1, 2, 3, ..., 1, 1, 1],
       [2, 0, 1, ..., 3, 0, 0],
       [1, 3, 1, ..., 3, 0, 0]])

In [63]:
end_index

array([[7, 7, 7, ..., 6, 7, 8],
       [7, 7, 7, ..., 5, 5, 8],
       [7, 6, 7, ..., 7, 5, 5],
       ...,
       [6, 8, 8, ..., 5, 5, 6],
       [7, 5, 8, ..., 7, 8, 8],
       [7, 6, 8, ..., 7, 7, 6]])

In [76]:
np.r_[map(tuple, np.vectorize(slice)(start_index, end_index))]

array([[slice(3, 7, None), slice(1, 7, None), slice(0, 7, None), ...,
        slice(2, 6, None), slice(2, 7, None), slice(1, 8, None)],
       [slice(0, 7, None), slice(2, 7, None), slice(1, 7, None), ...,
        slice(3, 5, None), slice(0, 5, None), slice(1, 8, None)],
       [slice(1, 7, None), slice(2, 6, None), slice(1, 7, None), ...,
        slice(0, 7, None), slice(3, 5, None), slice(1, 5, None)],
       ...,
       [slice(1, 6, None), slice(2, 8, None), slice(3, 8, None), ...,
        slice(1, 5, None), slice(1, 5, None), slice(1, 6, None)],
       [slice(2, 7, None), slice(0, 5, None), slice(1, 8, None), ...,
        slice(3, 7, None), slice(0, 8, None), slice(0, 8, None)],
       [slice(1, 7, None), slice(3, 6, None), slice(1, 8, None), ...,
        slice(3, 7, None), slice(0, 7, None), slice(0, 6, None)]],
      dtype=object)

In [69]:
np.r_[tuple(list(map(np.vectorize(slice), start_index, end_index)))]

array([slice(3, 7, None), slice(1, 7, None), slice(0, 7, None), ...,
       slice(3, 7, None), slice(0, 7, None), slice(0, 6, None)],
      dtype=object)

In [173]:
import pandas as pd
import numpy as np

df = pd.DataFrame([{'name': 'John', 'points' : [1,1,3,5]},{'name': 'John', 'points' : [2,0,1,5]},{'name': 'John', 'points' : [4,1,2,2]}])
df['points'] = df['points'].apply(lambda x : np.array(x)) # converting the list column to pd.array()

df

,name,points
0,John,"[1, 1, 3, 5]"
1,John,"[2, 0, 1, 5]"
2,John,"[4, 1, 2, 2]"


In [174]:
   name         points
0  John  [4, 1, 3, 5]

SyntaxError: invalid syntax (<ipython-input-174-35af9da025f7>, line 1)

In [221]:
df.groupby('name')['points'].apply(pd.Series)

0    [1, 1, 3, 5]
1    [2, 0, 1, 5]
2    [4, 1, 2, 2]
Name: points, dtype: object

In [282]:
import pandas as pd
df = pd.DataFrame({'a': [1,2,3,4,5,6,7,8,9,10], 'b': [11,22,11,22,33,11,22,44,11,22]})
df

,a,b
0,1,11
1,2,22
2,3,11
3,4,22
4,5,33
5,6,11
6,7,22
7,8,44
8,9,11
9,10,22


In [283]:
pd.crosstab(df['a'],df['b'])

b,11,22,33,44
a,,,,
1,1,0,0,0
2,0,1,0,0
3,1,0,0,0
4,0,1,0,0
5,0,0,1,0
6,1,0,0,0
7,0,1,0,0
8,0,0,0,1
9,1,0,0,0


In [284]:
df['a'].dot(df['b'])

1232

In [298]:
df

,a,b
0,1,11
1,2,22
2,3,11
3,4,22
4,5,33
5,6,11
6,7,22
7,8,44
8,9,11
9,10,22


In [315]:
print(df.assign(v=1).pivot('a','b').fillna(0))

      v               
b    11   22   33   44
a                     
1   1.0  0.0  0.0  0.0
2   0.0  1.0  0.0  0.0
3   1.0  0.0  0.0  0.0
4   0.0  1.0  0.0  0.0
5   0.0  0.0  1.0  0.0
6   1.0  0.0  0.0  0.0
7   0.0  1.0  0.0  0.0
8   0.0  0.0  0.0  1.0
9   1.0  0.0  0.0  0.0
10  0.0  1.0  0.0  0.0


In [320]:
a, b = 123, 4567

int(a*10**np.ceil(np.log10(b))+b)

1234567

1234567

1234567.0

In [321]:
import numpy as np

a, b = 123, 4567

#int(a*10**np.ceil(np.log10(b))+b)

In [363]:
def get_pos_nums(num):
    pos_nums = []
    n = 0
    while num != 0:
        pos_nums.append((num % 10) * 10**n)
        num = num // 10
        n = n + 1
    return pos_nums

In [367]:
def get_pos_nums(num):
    pos_nums = []
    while num != 0:
        pos_nums.append((num % 10))
        num = num // 10
    return pos_nums

In [370]:
digits_a = get_pos_nums(a)

In [371]:
digits_b = get_pos_nums(b)

In [372]:
digits_a

[3, 2, 1]

In [382]:
np.max([np.ceil(np.log10(b)), np.ceil(np.log10(a))])

4.0

In [405]:
from itertools import zip_longest

def get_pos_nums(num):
    pos_nums = []
    while num != 0:
        pos_nums.append((num % 10))
        num = num // 10
    return list(reversed(pos_nums))

l = [i for j in zip_longest(get_pos_nums(a), get_pos_nums(b)) for i in j if i!=None]
number = sum(d * 10**i for i, d in enumerate(l[::-1]))

1425367

In [373]:
digits_b

[7, 6, 5, 4]

In [377]:
power = max(len(digits_a), len(digits_b))

In [406]:
matrix = [[2,10,9,6],
          [5,1,4,7],
          [3,2,1,0], 
          [10, 20, 1, 4], 
          [17, 3, 5, 18]]

In [407]:
from scipy.spatial.distance import cdist
dist = cdist( matrix, matrix, metric='euclidean')

In [408]:
dist

array([[ 0.        , 10.77032961, 12.84523258, 15.23154621, 20.83266666],
       [10.77032961,  0.        ,  7.93725393, 20.09975124, 16.43167673],
       [12.84523258,  7.93725393,  0.        , 19.72308292, 23.17326045],
       [15.23154621, 20.09975124, 19.72308292,  0.        , 23.4520788 ],
       [20.83266666, 16.43167673, 23.17326045, 23.4520788 ,  0.        ]])

In [424]:
np.where(dist.argsort(1).argsort(1) <= 2, dist, 0)

array([[ 0.        , 10.77032961, 12.84523258,  0.        ,  0.        ],
       [10.77032961,  0.        ,  7.93725393,  0.        ,  0.        ],
       [12.84523258,  7.93725393,  0.        ,  0.        ,  0.        ],
       [15.23154621,  0.        , 19.72308292,  0.        ,  0.        ],
       [20.83266666, 16.43167673,  0.        ,  0.        ,  0.        ]])

In [433]:
import time

def f1(x):
    print("f1")
    print(x)
    time.sleep(2)
    return 0

def f2(x):
    print("f2")
    print(x)
    time.sleep(2)
    return 1


a = np.array([-1, 1])
np.where(a<0)

(array([0]),)

In [434]:
x = np.array([3, 4, 2, 1])

In [435]:
x

array([3, 4, 2, 1])

In [437]:
x[np.argpartition(x, 3)]

array([2, 1, 3, 4])

In [438]:
dist

array([[ 0.        , 10.77032961, 12.84523258, 15.23154621, 20.83266666],
       [10.77032961,  0.        ,  7.93725393, 20.09975124, 16.43167673],
       [12.84523258,  7.93725393,  0.        , 19.72308292, 23.17326045],
       [15.23154621, 20.09975124, 19.72308292,  0.        , 23.4520788 ],
       [20.83266666, 16.43167673, 23.17326045, 23.4520788 ,  0.        ]])

In [444]:
np.argpartition(dist, (0,1,2), axis=1)

array([[0, 1, 2, 3, 4],
       [1, 2, 0, 3, 4],
       [2, 1, 0, 3, 4],
       [3, 0, 2, 1, 4],
       [4, 1, 0, 3, 2]])

In [445]:
dist.argsort(1).argsort(1)

array([[0, 1, 2, 3, 4],
       [2, 0, 1, 4, 3],
       [2, 1, 0, 3, 4],
       [1, 3, 2, 0, 4],
       [2, 1, 3, 4, 0]])

In [448]:
dist

array([[ 0.        , 10.77032961, 12.84523258, 15.23154621, 20.83266666],
       [10.77032961,  0.        ,  7.93725393, 20.09975124, 16.43167673],
       [12.84523258,  7.93725393,  0.        , 19.72308292, 23.17326045],
       [15.23154621, 20.09975124, 19.72308292,  0.        , 23.4520788 ],
       [20.83266666, 16.43167673, 23.17326045, 23.4520788 ,  0.        ]])

In [452]:
np.argsort(-dist, axis=1)

array([[4, 3, 2, 1, 0],
       [3, 4, 0, 2, 1],
       [4, 3, 0, 1, 2],
       [4, 1, 2, 0, 3],
       [3, 2, 0, 1, 4]])

In [455]:
np.argpartition(np.argpartition(dist, (0,1,2)), (0,1,2))

array([[0, 1, 2, 3, 4],
       [2, 0, 1, 3, 4],
       [2, 1, 0, 3, 4],
       [1, 3, 2, 0, 4],
       [2, 1, 4, 3, 0]])

In [458]:
dist

array([[ 0.        , 10.77032961, 12.84523258, 15.23154621, 20.83266666],
       [10.77032961,  0.        ,  7.93725393, 20.09975124, 16.43167673],
       [12.84523258,  7.93725393,  0.        , 19.72308292, 23.17326045],
       [15.23154621, 20.09975124, 19.72308292,  0.        , 23.4520788 ],
       [20.83266666, 16.43167673, 23.17326045, 23.4520788 ,  0.        ]])

In [463]:
dist[dist.argsort(1)[-3:]]

array([[[12.84523258,  7.93725393,  0.        , 19.72308292,
         23.17326045],
        [10.77032961,  0.        ,  7.93725393, 20.09975124,
         16.43167673],
        [ 0.        , 10.77032961, 12.84523258, 15.23154621,
         20.83266666],
        [15.23154621, 20.09975124, 19.72308292,  0.        ,
         23.4520788 ],
        [20.83266666, 16.43167673, 23.17326045, 23.4520788 ,
          0.        ]],

       [[15.23154621, 20.09975124, 19.72308292,  0.        ,
         23.4520788 ],
        [ 0.        , 10.77032961, 12.84523258, 15.23154621,
         20.83266666],
        [12.84523258,  7.93725393,  0.        , 19.72308292,
         23.17326045],
        [10.77032961,  0.        ,  7.93725393, 20.09975124,
         16.43167673],
        [20.83266666, 16.43167673, 23.17326045, 23.4520788 ,
          0.        ]],

       [[20.83266666, 16.43167673, 23.17326045, 23.4520788 ,
          0.        ],
        [10.77032961,  0.        ,  7.93725393, 20.09975124,
         16

In [482]:
d = {'col1': [1,1,1,], 'col2': [2,2,2,]}
df = pd.DataFrame(data=d)

In [483]:
df[['col1']].to_numpy()

array([[1],
       [1],
       [1]])

In [473]:
arr = [np.array([1,2,3,4]),
       np.array([4,5,6,7]),
       np.array([8,9,10,11])]

#Getting the second column
np.stack(arr)[:,:2]

array([[1, 2],
       [4, 5],
       [8, 9]])

In [485]:
_failed_reason = target_info.get("failed_reason", None)
if _failed_reason==None:
    # string failed reason
    if isinstance(_failed_reason, str):
        task.failed_reason = _failed_reason

NameError: name 'target_info' is not defined

In [486]:
data = [[20779453, '2021-01-18 09:15:00Z', 62.47612], [20779453, '2021-01-18 09:20:00Z', 54.56400], 
[20779453, '2021-01-18 09:25:00Z', 64.95384], [20779453, '2021-01-18 09:30:00Z', 63.62500], 
[20779453, '2021-01-18 09:35:00Z', 61.51790]] 
df = pd.DataFrame(data, columns = ['ID', 'Timestamp', 'Value'])

In [487]:
df

,ID,Timestamp,Value
0,20779453,2021-01-18 09:15:00Z,62.47612
1,20779453,2021-01-18 09:20:00Z,54.56400
2,20779453,2021-01-18 09:25:00Z,64.95384
3,20779453,2021-01-18 09:30:00Z,63.62500
4,20779453,2021-01-18 09:35:00Z,61.51790


In [518]:
rolling = df['Value'].rolling(3)
flags = rolling.apply(lambda x: np.all(x>60))
consecutive_60 = sum(flags==1)*3
print(consecutive_60)

3


0    NaN
1    NaN
2    0.0
3    0.0
4    1.0
Name: Value, dtype: float64

In [542]:
from itertools import groupby
[len(list(g)) for k, g in groupby(df['Value']>60) if k==True][-1]

3

In [538]:
data = [[20779453, '2021-01-18 09:15:00Z', 62.47612], [20779453, '2021-01-18 09:20:00Z', 54.56400], 
[20779453, '2021-01-18 09:25:00Z', 64.95384], [20779453, '2021-01-18 09:30:00Z', 63.62500], 
[20779453, '2021-01-18 09:35:00Z', 61.51790], [20779453, '2021-01-18 09:40:00Z', 63.62500], 
[20779453, '2021-01-18 09:40:00Z', 53.62500],[20779453, '2021-01-18 09:45:00Z', 61.51790]] 
df1 = pd.DataFrame(data, columns = ['ID', 'Timestamp', 'Value'])

In [540]:
df1

,ID,Timestamp,Value
0,20779453,2021-01-18 09:15:00Z,62.47612
1,20779453,2021-01-18 09:20:00Z,54.56400
2,20779453,2021-01-18 09:25:00Z,64.95384
3,20779453,2021-01-18 09:30:00Z,63.62500
4,20779453,2021-01-18 09:35:00Z,61.51790
5,20779453,2021-01-18 09:40:00Z,63.62500
6,20779453,2021-01-18 09:40:00Z,53.62500
7,20779453,2021-01-18 09:45:00Z,61.51790


In [543]:
from itertools import groupby
[len(list(g)) for k, g in groupby(df1['Value']>60) if k==True][-1]

1

In [550]:
sum(df['Value'][::-1].expanding().apply(lambda x: np.all(x>60)))

3.0

In [561]:
df.set_index('ID')['Value'].idxmax()

20779453

In [558]:
df=.idxmax()

2

In [580]:
d = {'ID': {0: 20779453, 1: 20779453, 2: 20779456, 3: 20779453, 4: 20779453},
 'Timestamp': {0: '2021-01-18 09:15:00Z',
  1: '2021-01-18 09:20:00Z',
  2: '2021-01-18 09:25:00Z',
  3: '2021-01-18 09:30:00Z',
  4: '2021-01-18 09:35:00Z'},
 'Value': {0: 62.47612, 1: 54.564, 2: 64.95384, 3: 63.625, 4: 61.5179}}

In [593]:
dd = pd.DataFrame(d)[['ID','Value']]
dd

,ID,Value
0,20779453,62.47612
1,20779453,54.56400
2,20779456,64.95384
3,20779453,63.62500
4,20779453,61.51790


In [606]:
z = dd.groupby('ID').agg({'Value':['mean']}).reset_index()

In [610]:
z

,ID,Value
,,mean
0,60.545755,60.545755
1,64.953840,64.953840


In [591]:
dd.nlargest(2, 'Value')

,Value
ID,
20779456,64.95384
20779453,63.62500


In [611]:
l = [['a'],['b'],['cc']]
[i for sublist in l for i in sublist]

['a', 'b', 'cc']

['a', 'b', 'cc']

In [612]:
df = pd.read_clipboard()

In [614]:
df.to_dict()

{'row_label,': {0: 'prod_a,', 1: 'prod_b,', 2: 'prod_c,'},
 'jan,': {0: '100,', 1: '123,', 2: '10,'},
 'feb,': {0: '120,', 1: '233,', 2: '23,'},
 'mar,': {0: '123,', 1: '200,', 2: '34,'},
 'apr': {0: 150, 1: 40, 2: 33}}

In [647]:
df = {'row_label': {0: 'prod_a', 1: 'prod_b', 2: 'prod_c'},
 'jan': {0: '100', 1: '123', 2: '10'},
 'feb': {0: '120', 1: '233', 2: '23'},
 'mar': {0: '123', 1: '200', 2: '34'},
 'apr': {0: 150, 1: 40, 2: 33}}

In [648]:
df = pd.DataFrame(df)

In [649]:
df

,row_label,jan,feb,mar,apr
0,prod_a,100,120,123,150
1,prod_b,123,233,200,40
2,prod_c,10,23,34,33


In [660]:
prod_a,prod_b,prod_c = [pd.DataFrame(j[1:]).reset_index().set_axis(['row_label','value'], 1) for i,j in df.iterrows()]
print(prod_a)

  row_label value
0       jan   100
1       feb   120
2       mar   123
3       apr   150


In [701]:
idd = [1,1,1,1,1,2,2,2,2,2]
haz = [20,20,20,30,30,30,40,40,40,40]
prob = [0.05,0.05,0.42,0.3,0.02,0.05,0.12,0.44,0.05,0.05]
df = pd.DataFrame({'id':idd, 'haz':haz, 'prob':prob})

In [699]:
def f(l):
    return [i+(1-sum(l)) if i==max(l) else i for i in l]

df['new_proba'] = df.groupby(['id','haz'])['prob'].apply(lambda x: pd.Series(f(x))).values
print(df)

   id  haz  prob  new_proba
0   1   20  0.05       0.05
1   1   20  0.05       0.05
2   1   20  0.42       0.90
3   1   30  0.30       0.98
4   1   30  0.02       0.02
5   2   30  0.05       1.00
6   2   40  0.12       0.12
7   2   40  0.44       0.78
8   2   40  0.05       0.05
9   2   40  0.05       0.05


In [700]:
df.groupby(['id','haz'])['new_proba'].sum()

id  haz
1   20     1.0
    30     1.0
2   30     1.0
    40     1.0
Name: new_proba, dtype: float64

In [704]:
g = {'prob':['max','sum']}

In [710]:
df.groupby(['id','haz'])['prob'].transform('max')

0    0.42
1    0.42
2    0.42
3    0.30
4    0.30
5    0.05
6    0.44
7    0.44
8    0.44
9    0.44
Name: prob, dtype: float64

In [731]:
new_proba = df.groupby(['id','haz'])['prob'].apply(lambda x: max(x)+1-(sum(x))).values
df.loc[df.groupby(['id','haz'])['prob'].agg('idxmax').values, 'prob'] = new_proba

print(df)

   id  haz  prob
0   1   20  0.05
1   1   20  0.05
2   1   20  0.90
3   1   30  0.98
4   1   30  0.02
5   2   30  1.00
6   2   40  0.12
7   2   40  0.78
8   2   40  0.05
9   2   40  0.05

   id  haz  prob
0   1   20  0.05
1   1   20  0.05
2   1   20  0.90
3   1   30  0.98
4   1   30  0.02
5   2   30  1.00
6   2   40  0.12
7   2   40  0.78
8   2   40  0.05
9   2   40  0.05


,id,haz,prob
0,1,20,0.05
1,1,20,0.05
2,1,20,0.90
3,1,30,0.98
4,1,30,0.02
5,2,30,1.00
6,2,40,0.12
7,2,40,0.78
8,2,40,0.05
9,2,40,0.05


In [733]:
df = pd.read_clipboard()
df

,Column1,Column2,Column3
0,'cat','bat','xyz'
1,'toy','flower','abc'
2,'cat','bat','lmn'
3,'cat','bat','xyz'


In [734]:
df

,Column1,Column2,Column3
0,'cat','bat','xyz'
1,'toy','flower','abc'
2,'cat','bat','lmn'
3,'cat','bat','xyz'


In [739]:
print(df[~df.duplicated(keep='first')])

  Column1   Column2 Column3
0   'cat'     'bat'   'xyz'
1   'toy'  'flower'   'abc'
2   'cat'     'bat'   'lmn'


In [740]:
df.drop_duplicates(keep='first')

,Column1,Column2,Column3
0,'cat','bat','xyz'
1,'toy','flower','abc'
2,'cat','bat','lmn'


In [742]:
import pandas as pd

data = {'First':  ['First value','Third value','Second value','First value','Third value','Second value'],
        'Second': ['the old man is here','the young girl is there', 'the old woman is here','the  young boy is there','the young girl is here','the old girl is here']}

df = pd.DataFrame (data, columns = ['First','Second'])

In [743]:
df

,First,Second
0,First value,the old man is here
1,Third value,the young girl is there
2,Second value,the old woman is here
3,First value,the young boy is there
4,Third value,the young girl is here
5,Second value,the old girl is here


In [748]:
import itertools

In [768]:
all_pairs = [(i,l,similarity_measure(j,m)) for (i,j),(l,m) in 
             product(zip(df['First'], df['Second']), repeat=2) if i!=l]

df_pairs = pd.DataFrame(all_pairs, columns=['A','B','Similarity'])
pair_similarity = df_pairs.groupby(['A','B'])['Similarity'].mean().reset_index()
print(pair_similarity)

              A             B  Similarity
0   First value  Second value   68.571429
1   First value   Third value   74.107143
2  Second value   First value   68.571429
3  Second value   Third value   65.126050
4   Third value   First value   74.107143
5   Third value  Second value   65.126050

              A             B  Similarity
0   First value  Second value   68.571429
1   First value   Third value   74.107143
2  Second value   First value   68.571429
3  Second value   Third value   65.126050
4   Third value   First value   74.107143
5   Third value  Second value   65.126050


In [770]:
group_similarity = df_pairs.groupby(['A'])['Similarity'].mean().reset_index()
print(group_similarity)

              A  Similarity
0   First value   71.339286
1  Second value   66.848739
2   Third value   69.61659

              A  Similarity
0   First value   71.339286
1  Second value   66.848739
2   Third value   69.616597


In [773]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/akshay/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [774]:
import nltk
from itertools import combinations, product

#function to calculate similarity between each pairs of documents 
def similarity_measure(doc1, doc2): 

    words_doc1 = set(doc1) 
    words_doc2 = set(doc2)

    intersection = words_doc1.intersection(words_doc2)
    union = words_doc1.union(words_doc2)
    
    return float (len(intersection)) / len(union) * 100

#getting the lemmatized text along side the intents
data_similarity= df.groupby('First')['Second'].apply(lambda x:  nltk.tokenize.word_tokenize(' '.join(x)))
data_similarity = data_similarity.reset_index()

all_pairs = [(i,l,similarity_measure(j,m)) for (i,j),(l,m) in 
             product(zip(data_similarity['First'], data_similarity['Second']), repeat=2) if i!=l]

pair_similarity = pd.DataFrame(all_pairs, columns=['A','B','Similarity'])
group_similarity = pair_similarity.groupby(['A'])['Similarity'].mean().reset_index()
print(group_similarity)

              A  Similarity
0   First value   47.777778
1  Second value   45.000000
2   Third value   52.777778

In [775]:
data_similarity

,First,Second
0,First value,"[the, old, man, is, here, the, young, boy, is,..."
1,Second value,"[the, old, woman, is, here, the, old, girl, is..."
2,Third value,"[the, young, girl, is, there, the, young, girl..."


In [783]:
all_pairs

[('First value', 'Second value', 40.0),
 ('First value', 'Third value', 55.55555555555556),
 ('Second value', 'First value', 40.0),
 ('Second value', 'Third value', 50.0),
 ('Third value', 'First value', 55.55555555555556),
 ('Third value', 'Second value', 50.0)]

              A  Similarity
0   First value   47.777778
1  Second value   45.000000
2   Third value   52.777778


In [796]:
all_data = [['John','Steven','Mosh'],
           ['Juan','Mata','De Gea']]

names = []

for sublist in all_data:
    for string in sublist:
        if string.count("e")>=2:
            names.append(string)
names

['Steven', 'De Gea']

In [797]:
[string for sublist in all_data for string in sublist if string.count("e")>=2]

['Steven', 'De Gea']

In [845]:
data =[{'ID': '11348',
      'NAME': 'Yanara',
      'LAST_NAME': 'Araneda',
      'LEAD_ID': '17772',
      'EMAIL': [{'ID': '42400',
        'VALUE_TYPE': 'WORK',
        'VALUE': 'yanara.araneda.moraga@gmail.com',
        'TYPE_ID': 'EMAIL'}]},
     {'ID': '11346',
      'NAME': 'Carlos ',
      'LAST_NAME': 'Gonzalez ',
      'LEAD_ID': '17782',
      'EMAIL': [{'ID': '42390',
        'VALUE_TYPE': 'WORK',
        'VALUE': 'cmgclima@gmail.com',
        'TYPE_ID': 'EMAIL'}]},
     {'ID': '11344',
      'NAME': 'Alcides',
      'LAST_NAME': 'Gonzalez',
      'LEAD_ID': '11006',
      'EMAIL': [{'ID': '42376',
        'VALUE_TYPE': 'WORK',
        'VALUE': 'makogonzalez@gmail.com',
        'TYPE_ID': 'EMAIL'}]}]

p1 = pd.json_normalize(data).drop('EMAIL', axis=1)
p2 = pd.json_normalize(data, record_path='EMAIL', record_prefix='EMAIL.')

df = p1.merge(p2, left_index=True, right_index=True)
print(df)

      ID     NAME  LAST_NAME LEAD_ID EMAIL.ID EMAIL.VALUE_TYPE  \
0  11348   Yanara    Araneda   17772    42400             WORK   
1  11346  Carlos   Gonzalez    17782    42390             WORK   
2  11344  Alcides   Gonzalez   11006    42376             WORK   

                       EMAIL.VALUE EMAIL.TYPE_ID  
0  yanara.araneda.moraga@gmail.com         EMAIL  
1               cmgclima@gmail.com         EMAIL  
2           makogonzalez@gmail.com         EMAIL  


In [844]:
data =[{'ID': '11348',
      'NAME': 'Yanara',
      'LAST_NAME': 'Araneda',
      'LEAD_ID': '17772',
      'EMAIL': {'ID': '42400',
        'VALUE_TYPE': 'WORK',
        'VALUE': 'yanara.araneda.moraga@gmail.com',
        'TYPE_ID': 'EMAIL'}},
     {'ID': '11346',
      'NAME': 'Carlos ',
      'LAST_NAME': 'Gonzalez ',
      'LEAD_ID': '17782',
      'EMAIL': {'ID': '42390',
        'VALUE_TYPE': 'WORK',
        'VALUE': 'cmgclima@gmail.com',
        'TYPE_ID': 'EMAIL'}},
     {'ID': '11344',
      'NAME': 'Alcides',
      'LAST_NAME': 'Gonzalez',
      'LEAD_ID': '11006',
      'EMAIL': {'ID': '42376',
        'VALUE_TYPE': 'WORK',
        'VALUE': 'makogonzalez@gmail.com',
        'TYPE_ID': 'EMAIL'}}]

print(pd.json_normalize(data))

      ID     NAME  LAST_NAME LEAD_ID EMAIL.ID EMAIL.VALUE_TYPE  \
0  11348   Yanara    Araneda   17772    42400             WORK   
1  11346  Carlos   Gonzalez    17782    42390             WORK   
2  11344  Alcides   Gonzalez   11006    42376             WORK   

                       EMAIL.VALUE EMAIL.TYPE_ID  
0  yanara.araneda.moraga@gmail.com         EMAIL  
1               cmgclima@gmail.com         EMAIL  
2           makogonzalez@gmail.com         EMAIL  


In [870]:
df['Stock'].str.split('.', expand = True)[0]

0      IBGL
0      EHYA
0      HIGH
0      IHYG
0      HYGU
0      IBCI
0      IAEX
0      AGED
0      ISAG
0      IAPD
0      IASP
0      RBOT
0      RBOD
0      BRIC
0      CNYB
0       FXC
0     DJSXE
0    CSSX5E
Name: 0, dtype: object

In [846]:
df = pd.read_clipboard()

In [869]:
.str.split('.').str.get(0)

0      IBGL
0      EHYA
0      HIGH
0      IHYG
0      HYGU
0      IBCI
0      IAEX
0      AGED
0      ISAG
0      IAPD
0      IASP
0      RBOT
0      RBOD
0      BRIC
0      CNYB
0       FXC
0     DJSXE
0    CSSX5E
Name: Stock, dtype: object

In [873]:
[i for i in reversed(v)]

[7, 7, 0, 6, 6, 0, 5, 5, 0]

In [876]:
s = [0]
[s.append(s[0]+n) or s.pop(0) for n in v]

[0, 0, 5, 10, 10, 16, 22, 22, 29]

In [878]:
for x in mylist:
    (bad, good)[x in goodvals].append(x)

NameError: name 'mylist' is not defined

In [902]:
[list(v) for k,v in itertools.groupby(reversed(v))]

[[7, 7], [0], [6, 6], [0], [5, 5], [0]]

In [923]:
from itertools import accumulate

acc = list(accumulate(reversed(v)))
sums = [j if i==0 else i for i,j in zip(reversed(v), acc)]
out = list(reversed(sums))
print(out)

[36, 5, 5, 26, 6, 6, 14, 7, 7]

[36, 5, 5, 26, 6, 6, 14, 7, 7]


In [929]:
%%timeit
list(reversed([j if i==0 else i for i,j in zip(reversed(v), accumulate(reversed(v)))]))

1.6 µs ± 43.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [930]:
%%timeit
for i in range(len(v) - 1, -1, -1):       
    if v[i] == 0 :
        v[i] = sum(v[i+1:i+4])

744 ns ± 30.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [933]:
[0]

TypeError: 'list_reverseiterator' object is not subscriptable

In [948]:
out = []
v = [0, 5, 5, 0, 6, 6, 0, 7, 7 ]
vr = list(reversed(v))

for i in range(len(vr)):
    if vr[i]==0:
        out.append(sum(vr[0:i]))
    else:
        out.append(vr[i])

out

[7, 7, 14, 6, 6, 26, 5, 5, 36]

In [942]:
vr

[7, 7, 14, 6, 6, 26, 5, 5, 36]

In [964]:
x = False

[x and 'ai' if x else continue]

SyntaxError: invalid syntax (<ipython-input-964-e8f0b6be18ff>, line 3)

In [1050]:
data1 = 'hi'
data2 = 'there'

x, y = True, False

[i for i in [data1 if x else None, data2 if y else None] if i]

['hi']

['hi']

In [1047]:
[i for i in [data1 if x else None, data2 if y else None] if i]

['there']

In [1119]:
np.array([data1, data2])[[x,y]].tolist()

['hi']

In [1120]:
x, y = True, False
dict(np.array(list(d.items()))[[x,y]])

{'data1': 'hi'}

In [1122]:
import random

In [1123]:
training_data=[]
for i in range(0,5):
    training_data.append([random.uniform(0,0.2944)])
for i in range(0,5):
    training_data.append([random.uniform(0.2944,1.7394)])
for i in range(0,5):
    training_data.append([random.uniform(1.7394,3.2394)])
for i in range(0,5):
    training_data.append([random.uniform(3.2394,6)])

In [1126]:
np.array(training_data).shape

(20, 1)

In [1084]:
# dictionary with data
d = {'userid':[100,100,100,101,101],
     'col_a':[1,3,3,np.NaN,4],
    'col_b':[2,5,5,4,4],
    'col_c':[3,4,5,1,np.NaN]}
# create dataframe
df = pd.DataFrame(d)
# make the userid the index
df = df.set_index('userid')

In [1085]:
df.mean(1).groupby('userid').mean()

userid
100    3.444444
101    3.250000
dtype: float64

In [1143]:
a = np.arange(0,10).reshape(5,2)
b = np.arange(0,10).reshape(5,2)

c = np.c_[a,b]
np.random.shuffle(c)

a1, b1 = c[:,:a.shape[1]], c[:,:b.shape[1]]

print(a1)

[[8 9]
 [2 3]
 [4 5]
 [6 7]
 [0 1]]


In [1144]:
foo = pd.to_datetime(pd.Series([
    '2020-01-01 12:00:00',
    '2020-02-02 23:12:00'
]))

bar = pd.to_datetime(pd.Series([
    '2020-01-20 01:02:03',
    '2020-01-30 03:02:01'
]))

0   2020-01-01 12:00:00
1   2020-02-02 23:12:00
dtype: datetime64[ns]

In [1146]:
bar

0   2020-01-20 01:02:03
1   2020-01-30 03:02:01
dtype: datetime64[ns]

In [1161]:
df = pd.read_clipboard('\s\s+')
df

,date,open,high,low,close,a_column
0,2007-01-01 00:00,0.78941,0.79062,0.78693,0.79000,0.0
1,2007-01-02 00:00,0.78998,0.79700,0.78950,0.79548,0.0
2,2007-01-03 00:00,0.79527,0.79790,0.78988,0.79140,0.0
3,2007-01-04 00:00,0.79138,0.79200,0.78197,0.78270,0.0
4,2007-01-05 00:00,0.78267,0.78515,0.77761,0.77815,0.0


In [1162]:
dfBi = df.query('a_column>0').index
normal = []
n_future = []

In [1163]:
# for each number in the index variable
for x in dfBi:
    #conditions
    if (df['close'].loc[x] < df['close'].loc[x+1] and df['close'].loc[x+1] > df['close'].loc[x+2]):

        #add that index where the condition is true to a list
        normal.append(x), 

        #add that index where the condition is true and data in future (from the index position)
        n_future.append(x),n_future.append(x+1),n_future.append(x+2),n_future.append(x+3)

In [1164]:
normal

[]

In [1166]:
common_keywords = ['dog', 'person', 'cat']

In [1165]:
people = [{'name':'Bob', 'keywords': ['dog', 'dog', 'car', 'trampoline']},
          {'name':'Kate', 'keywords': ['cat', 'jog', 'tree', 'flower']},
           {'name':'Sasha', 'keywords': ['cooking', 'stove', 'person', 'cat']}]

In [1198]:
from collections import Counter

[{'name':i.get('name'),
  'keywords':[dict(Counter([j for j in i.get('keywords') 
                            if j in common_keywords]))]} for i in people]

[{'name': 'Bob', 'keywords': [{'dog': 2}]},
 {'name': 'Kate', 'keywords': [{'cat': 1}]},
 {'name': 'Sasha', 'keywords': [{'person': 1, 'cat': 1}]}]

[{'name': 'Bob', 'keywords': {'dog': 2}},
 {'name': 'Kate', 'keywords': {'cat': 1}},
 {'name': 'Sasha', 'keywords': {'person': 1, 'cat': 1}}]

In [1212]:
#Dummh series
d = ['{58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5}', 
     '{58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5}', 
     '{58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5 |58.5}']
dd = pd.Series(d)

out = dd.str[1:-1].str.split("|",expand=True).add_prefix("setrwds_x")
print(out)

  setrwds_x0 setrwds_x1 setrwds_x2 setrwds_x3 setrwds_x4 setrwds_x5  \
0      58.5       58.5       58.5       58.5       58.5       58.5    
1      58.5       58.5       58.5       58.5       58.5       58.5    
2      58.5       58.5       58.5       58.5       58.5       58.5    

  setrwds_x6 setrwds_x7 setrwds_x8 setrwds_x9 setrwds_x10 setrwds_x11  
0      58.5       58.5       58.5       58.5        58.5         58.5  
1      58.5       58.5       58.5       58.5        58.5         58.5  
2      58.5       58.5       58.5       58.5        58.5         58.5

  setrwds_x0 setrwds_x1 setrwds_x2 setrwds_x3 setrwds_x4 setrwds_x5  \
0      58.5       58.5       58.5       58.5       58.5       58.5    
1      58.5       58.5       58.5       58.5       58.5       58.5    
2      58.5       58.5       58.5       58.5       58.5       58.5    

  setrwds_x6 setrwds_x7 setrwds_x8 setrwds_x9 setrwds_x10 setrwds_x11  
0      58.5       58.5       58.5       58.5        58.5         58.5  
1      58.5       58.5       58.5       58.5        58.5         58.5  
2      58.5       58.5       58.5       58.5        58.5         58.5  


In [1211]:
dd.str[1:-1].str.split("|",expand=True).add_prefix("setrwds_x")

,setrwds_x0,setrwds_x1,setrwds_x2,setrwds_x3,setrwds_x4,setrwds_x5,setrwds_x6,setrwds_x7,setrwds_x8,setrwds_x9,setrwds_x10,setrwds_x11
0,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5
1,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5
2,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5


In [1206]:
dd.str.split("|",expand=True).add_prefix("setrwds_x")

,setrwds_x0,setrwds_x1,setrwds_x2,setrwds_x3,setrwds_x4,setrwds_x5,setrwds_x6,setrwds_x7,setrwds_x8,setrwds_x9,setrwds_x10,setrwds_x11
0,{58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5}
1,{58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5}
2,{58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5,58.5}


In [1233]:
docs =  [
{'isActive': 1, 'description': 'teleconference call.\n\n'}, 
{'isActive': 1, 'description': 'calls to review capacity.\n'}, 
{'isActive': 1, 'description': 'communications \r\n.'}
]

In [1234]:
df = pd.json_normalize(docs)
df['description'] = df['description'].str.strip().str.replace(" \r\n","")
print(df)

   isActive                description
0         1       teleconference call.
1         1  calls to review capacity.
2         1            communications.


In [1248]:
d = [{'isActive': True, 'pressure': 2, 'actions': [{'isActive': True, 'description': 'Ensure  place in Majors\r\n'}]},
     {'isActive': True, 'pressure': 2, 'actions': [{'isActive': True, 'description': 'Ensure  place in Majors\r\n'}]},
     {'isActive': True, 'pressure': 2, 'actions': [{'isActive': True, 'description': 'Ensure  place in Majors\r\n'}]}]

In [1249]:
d

[{'isActive': True,
  'pressure': 2,
  'actions': [{'isActive': True,
    'description': 'Ensure  place in Majors\r\n'}]},
 {'isActive': True,
  'pressure': 2,
  'actions': [{'isActive': True,
    'description': 'Ensure  place in Majors\r\n'}]},
 {'isActive': True,
  'pressure': 2,
  'actions': [{'isActive': True,
    'description': 'Ensure  place in Majors\r\n'}]}]

In [1239]:
data =[{'ID': '11348',
      'NAME': 'Yanara',
      'LAST_NAME': 'Araneda',
      'LEAD_ID': '17772',
      'EMAIL': [{'ID': '42400',
        'VALUE_TYPE': 'WORK',
        'VALUE': 'yanara.araneda.moraga@gmail.com',
        'TYPE_ID': 'EMAIL'}]},
     {'ID': '11346',
      'NAME': 'Carlos ',
      'LAST_NAME': 'Gonzalez ',
      'LEAD_ID': '17782',
      'EMAIL': [{'ID': '42390',
        'VALUE_TYPE': 'WORK',
        'VALUE': 'cmgclima@gmail.com',
        'TYPE_ID': 'EMAIL'}]},
     {'ID': '11344',
      'NAME': 'Alcides',
      'LAST_NAME': 'Gonzalez',
      'LEAD_ID': None,
     }]

In [1270]:
cols = pd.MultiIndex.from_arrays([['A','B','C'],['a','b','c']])
df = pd.DataFrame(np.random.randint(0,10,(6,3)), columns=cols)
df

,A,B,C
,a,b,c
0,3,7,5
1,5,8,0
2,8,3,8
3,2,9,2
4,7,9,5
5,1,8,3


In [1273]:
df = df.drop('A',axis=1, level=0)
df['a']=[1,2,3,4,5,6]

In [1274]:
df

,B,C,a
,b,c,
0,7,5,1
1,8,0,2
2,3,8,3
3,9,2,4
4,9,5,5
5,8,3,6


In [1275]:
df.columns = df.columns.map('|'.join).str.strip('|')

In [1276]:
df.columns

Index(['B|b', 'C|c', 'a'], dtype='object')

In [1277]:
df['D|d'] = df['a'].astype(str) + '-' + df['C|c'].astype(str)

In [1278]:
df

,B|b,C|c,a,D|d
0,7,5,1,1-5
1,8,0,2,2-0
2,3,8,3,3-8
3,9,2,4,4-2
4,9,5,5,5-5
5,8,3,6,6-3


In [1296]:
my_index = pd.MultiIndex.from_arrays([['A','B',None],['a','b','c'],['1','2','3']])
my_index

MultiIndex([('A', 'a', '1'),
            ('B', 'b', '2'),
            (nan, 'c', '3')],
           )

In [1298]:
df = pd.DataFrame(np.random.randint(0,5,(10,3)),columns=my_index)
#df = df.set_index(['one', 'two', 'three'])
df

,A,B,NaN
,a,b,c
,1,2,3
0,4,0,2
1,3,4,3
2,1,1,0
3,2,3,4
4,1,3,0
5,4,2,4
6,4,4,0
7,0,1,3


In [1281]:
d = pd.DataFrame(np.random.random((40,202)))

In [1285]:
d.columns = d.columns.astype(str)

In [1286]:
d.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '192', '193', '194', '195', '196', '197', '198', '199', '200', '201'],
      dtype='object', length=202)

In [1299]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [1306]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]], =[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 2),
            ('b', 2)],
           )

In [1265]:
pd.Index(['IDRSSD', 'RCFD3531|TRDG ASSETS-US TREAS SECS IN DOM OFF',
       'RCFD3532|TRDG ASSETS-US GOV AGC CORP OBLGS',
       'RCFD3533|TRDG ASSETS-SECS ISSD BY ST  POL SUB',
       'TEXTF660|3RD ITEMIZED AMT FOR OTHR TRDG ASTS',
       'Unnamed: 115_level_0|Unnamed: 115_level_1',
       'Unnamed: 133_level_0|Unnamed: 133_level_1',
       'Unnamed: 139_level_0|Unnamed: 139_level_1',
       'Unnamed: 20_level_0|Unnamed: 20_level_1',
       'Unnamed: 87_level_0|Unnamed: 87_level_1', 'file', 'schedule_code',
       'year', 'qyear'],
      dtype='object', length=202)

TypeError: Unexpected keyword arguments {'length'}

In [1308]:
df = pd.read_clipboard('\s\s+')

In [1348]:
d = {'Node': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
     'TLW': {0: [2, 22, 3], 1: [12], 2: [2,43], 3: [3], 4: [11]}}
df = pd.DataFrame(d)


new_df = pd.DataFrame(np.unique(df.explode('TLW').values), columns=['Nodes'])
print(new_df)

  Nodes
0     1
1     2
2     3
3     4
4     5
5    11
6    12
7    22
8    43


In [1349]:
d = {'Node': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5},
     'TLW': {0: [2, 22, 3], 1: [12], 2: [2,43], 3: [3], 4: [11]}}
df = pd.DataFrame(d)

nodes = df.apply(lambda x: [x['Node']]+ x['TLW'], axis=1).explode().unique()
new_df = pd.DataFrame(nodes, columns=['Node'])
print(new_df)

  Node
0    1
1    2
2   22
3    3
4   12
5   43
6    4
7    5
8   11


In [1396]:
    X     Y
0   a     1
1   b    10
2   c    11
3   d   100
4   e   101
5   f   110
6   g   122

SyntaxError: invalid syntax (<ipython-input-1396-ceb61ccd69f7>, line 1)

In [1400]:
df = pd.read_clipboard('\s\s+')

In [1401]:
df

,X,Y
0,a,1
1,b,10
2,c,11
3,d,100
4,e,101
5,f,110
6,g,122


In [1402]:
decompose = lambda x: [int(j)*10**i for i,j in enumerate(str(x)[::-1]) if int(j)]

df['Y'] = df['Y'].apply(decompose)
out = df.explode('Y')
print(out)

   X    Y
0  a    1
1  b   10
2  c    1
2  c   10
3  d  100
4  e    1
4  e  100
5  f   10
5  f  100
6  g    2
6  g   20
6  g  100


In [1374]:
'abcd'

'dcba'

In [1381]:
decompose = lambda x: [int(j)*10**i for i,j in enumerate(str(x)[::-1]) if int(j)!=0]

In [1384]:
decompose(111)

[1, 10, 100]

[10, 100]

In [1407]:
thisdict = {
  "brand": "Ford",
  "model": [1,4,9,16],
  "year": 1964
}

In [1423]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline
import xgboost as xgb

X = np.random.random(size=(1000, 5))
y = np.random.choice([0,1], 1000)
w = np.random.random(size=(1000,))

scaler = StandardScaler()
log_reg = LogisticRegression()

params = {
    'n_estimators': 10,
    'max_depth': 3,
    'learning_rate': 0.1
}

log_reg_pipe = make_pipeline(
    scaler,
    log_reg
)

stack_pipe = make_pipeline(
    StackingClassifier(
        estimators=[('lr', log_reg_pipe)],
        final_estimator=xgb.XGBRegressor(**params),
        passthrough=True,
        cv=2
    )
)

In [1429]:
x = np.random.random((4, 3, 32, 32))

np.add.reduce(x,1,keepdims=True).shape
x.sum(1, keepdims=True).shape

(4, 1, 32, 32)

In [1449]:
df = pd.DataFrame({'A':['a','b','a','a','b'],'B':[100,100,120,150,200]})

df['subindex'] = df.groupby('A')['B'].rank(method='min')
df = df.sort_values(['A','subindex'])
print(df)

   A    B  subindex
0  a  100       1.0
2  a  120       2.0
3  a  150       3.0
1  b  100       1.0
4  b  200       2.0

   A    B  subindex
0  a  100       1.0
2  a  120       2.0
3  a  150       3.0
1  b  100       1.0
4  b  200       2.0


,A,B,subindex
0,a,100,1.0
2,a,120,2.0
3,a,150,3.0
1,b,100,1.0
4,b,200,2.0


In [1450]:
l = ['Nimzowitsch Defense: Kennedy Variation',
     'Kings Pawn Game: Leonardis Variation',
     'Queens Pawn Game: Zukertort Variation',
     'Philidor Defense']

l = pd.Series(l)

l.str.split(':').str[0]

In [1453]:
l.str.split(':').str[0]

0    Nimzowitsch Defense
1        Kings Pawn Game
2       Queens Pawn Game
3       Philidor Defense
dtype: object

In [1458]:
x = int(1)
y = int(1)
z = int(1)
n = int(2)
result = [[i, j, k] for i in range(n) for j in range(n) for k in range(n) if n != sum([i, j, k])]

In [1459]:
result

[[0, 0, 0], [0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1]]

In [1460]:
list1 = [1,10,100]

In [1466]:
[j for i in list1 for j in [i,i+1,i+2]]

[1, 2, 3, 10, 11, 12, 100, 101, 102]

In [1522]:
names = ['Kenny Rogers', 'Reba McEntire', 'Johnny Cash', 'Tito Jackson', 'Tzuyu', 'Kareena Kapoor']
dirname = ['/home/users/krogers', '/home/rmcentire', '/home/users/jcash', '/home/hut/titoj', 
'/home/users/tzuyu', '/home/users2/kkapoor']

mname = max([len(i) for i in names])
mdir = max([len(i) for i in dirname])


print("+----------------+-----------------------+")
print("|", names[0].ljust(mname), "|", dirname[0].ljust(mdir), " |")
print("|", names[1].ljust(mname), "|", dirname[1].ljust(mdir), " |")
print("|", names[2].ljust(mname), "|", dirname[2].ljust(mdir), " |")
print("|", names[3].ljust(mname), "|", dirname[3].ljust(mdir), " |")
print("|", names[4].ljust(mname), "|", dirname[4].ljust(mdir), " |")
print("|", names[5].ljust(mname), "|", dirname[5].ljust(mdir), " |")
print("+----------------+-----------------------+")


print('+'+'-'*(mname+2)+'+'+'-'*(mdir+3)+'+')
for i,j in zip(names, dirname):
    print("|", i.ljust(mname), "|", j.ljust(mdir), " |")
print('+'+'-'*(mname+2)+'+'+'-'*(mdir+3)+'+')

+----------------+-----------------------+
| Kenny Rogers   | /home/users/krogers   |
| Reba McEntire  | /home/rmcentire       |
| Johnny Cash    | /home/users/jcash     |
| Tito Jackson   | /home/hut/titoj       |
| Tzuyu          | /home/users/tzuyu     |
| Kareena Kapoor | /home/users2/kkapoor  |
+----------------+-----------------------+
+----------------+-----------------------+
| Kenny Rogers   | /home/users/krogers   |
| Reba McEntire  | /home/rmcentire       |
| Johnny Cash    | /home/users/jcash     |
| Tito Jackson   | /home/hut/titoj       |
| Tzuyu          | /home/users/tzuyu     |
| Kareena Kapoor | /home/users2/kkapoor  |
+----------------+-----------------------+


In [1500]:
'plain', 'simple', 'grid', 'pipe', 'orgtbl', 'rst', 'mediawiki',
'latex', 'latex_raw' and 'latex_booktabs'

In [1516]:
#!pip install tabulate
from tabulate import tabulate
print(tabulate(zip(names, dirname), tablefmt="grid"))

print(tabulate(zip(names, dirname), tablefmt="orgtbl"))

print(tabulate(zip(names, dirname), tablefmt="rst"))

+----------------+----------------------+
| Kenny Rogers   | /home/users/krogers  |
+----------------+----------------------+
| Reba McEntire  | /home/rmcentire      |
+----------------+----------------------+
| Johnny Cash    | /home/users/jcash    |
+----------------+----------------------+
| Tito Jackson   | /home/hut/titoj      |
+----------------+----------------------+
| Tzuyu          | /home/users/tzuyu    |
+----------------+----------------------+
| Kareena Kapoor | /home/users2/kkapoor |
+----------------+----------------------+
| Kenny Rogers   | /home/users/krogers  |
| Reba McEntire  | /home/rmcentire      |
| Johnny Cash    | /home/users/jcash    |
| Tito Jackson   | /home/hut/titoj      |
| Tzuyu          | /home/users/tzuyu    |
| Kareena Kapoor | /home/users2/kkapoor |
==============  ====================
Kenny Rogers    /home/users/krogers
Reba McEntire   /home/rmcentire
Johnny Cash     /home/users/jcash
Tito Jackson    /home/hut/titoj
Tzuyu           /home/users/tzuyu
Kareena Kapoor  /home/users2/kkapoor
==============  ====================

+----------------+----------------------+
| Kenny Rogers   | /home/users/krogers  |
+----------------+----------------------+
| Reba McEntire  | /home/rmcentire      |
+----------------+----------------------+
| Johnny Cash    | /home/users/jcash    |
+----------------+----------------------+
| Tito Jackson   | /home/hut/titoj      |
+----------------+----------------------+
| Tzuyu          | /home/users/tzuyu    |
+----------------+----------------------+
| Kareena Kapoor | /home/users2/kkapoor |
+----------------+----------------------+
| Kenny Rogers   | /home/users/krogers  |
| Reba McEntire  | /home/rmcentire      |
| Johnny Cash    | /home/users/jcash    |
| Tito Jackson   | /home/hut/titoj      |
| Tzuyu          | /home/users/tzuyu    |
| Kareena Kapoor | /home/users2/kkapoor |
==============  ====================
Kenny Rogers    /home/users/krogers
Reba McEntire   /home/rmcentire
Johnny Cash     /home/users/jcash
Tito Jackson    /home/hut/titoj
Tzuyu           /home/users/tzu

In [1589]:
data = {'First':  [1, 2,3],
        'Second': [2,1,5],
         'Third': [5,1,2]
        }
df = pd.DataFrame (data, columns = ['First','Second','Third'])

In [1581]:
new_data = (df.to_numpy().argsort(0).argsort()>0).astype(int)
df2 = pd.DataFrame(new_data, columns = ['First','Second','Third'])
print(df2)

   First  Second  Third
0      0       1      1
1      1       0      1
2      1       0      1


In [1593]:
df

,First,Second,Third
0,1,2,5
1,2,1,1
2,3,5,2


In [1598]:
print((df.rank(axis=1, method='min')>1).astype(int))

   First  Second  Third
0      0       1      1
1      1       0      0
2      1       1      0


'01:22:10'

In [1527]:
time.mktime(z[0].split(':'))

TypeError: Tuple or struct_time argument required

In [1610]:
df

,First,Second,Third
0,1,2,5
1,2,1,1
2,3,5,2


In [1615]:
df.shape[1]-2

1

In [1633]:
n = 2
(df.rank(axis=1, method='min')>(df.shape[1]-n)).astype(int)

,First,Second,Third
0,0,1,1
1,1,0,0
2,1,1,0


In [1632]:

new_data

array([[0, 1, 1],
       [1, 0, 1],
       [1, 1, 0]])

In [1634]:
(df.to_numpy().argsort().argsort()>(df.shape[1]-n-1)).astype(int)

array([[0, 1, 1],
       [1, 0, 1],
       [1, 1, 0]])

In [1607]:
df.rank(axis=1, method='min')

,First,Second,Third
0,1.0,2.0,3.0
1,3.0,1.0,1.0
2,2.0,3.0,1.0


In [1652]:
df = pd.DataFrame({'A': [1.23456, 2.34567,3.45678],  'B':[2,3,4]})
df['A'] = df['A'].round(2)
df[['A']].style.bar(subset='A')


f = {'A':'{:.2f}'} #column a to 2 decimals
df.style.format(f).bar(subset='A')

,A,B
0,1.23,2
1,2.35,3
2,3.46,4


In [1657]:
data = {'First':  [1, 2,3],
        'Second': [2,1,5],
         'Third': [5,3,2]
        }
df = pd.DataFrame (data, columns = ['First','Second','Third'])

In [1658]:
df

,First,Second,Third
0,1,2,5
1,2,1,3
2,3,5,2


In [1675]:
(df.rank(1)>(df.shape[1]-2)).astype(int)

,First,Second,Third
0,0,1,1
1,1,0,1
2,1,1,0


In [1674]:
(df.to_numpy().argsort(1).argsort())#>(df.shape[1]-n-1)).astype(int)

array([[0, 1, 2],
       [1, 0, 2],
       [1, 2, 0]])

In [1682]:
n = 6
sample_size = 300
code_size = 50
learning_rate = 0.001
n_bags= None

# autoencoder: n_bags X n_instances_in_bag X n_samples (timesteps) X n_measurements
input_window = layers.Input(shape=(n_bags,sample_size, n)) 
x = layers.Conv1D(filters=40, kernel_size=21, activation='relu', padding='valid')(input_window)
#x = layers.MaxPooling1D(pool_size=2)(x)

x = layers.MaxPooling2D(pool_size=(1,2))(x)

model = Model(input_window, x)

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, None, 300, 6)]    0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 280, 40)     5080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, None, 140, 40)     0         
Total params: 5,080
Trainable params: 5,080
Non-trainable params: 0
_________________________________________________________________


In [1693]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Lambda, BatchNormalization, Conv2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

input_dim = (66,200,3)

model = Sequential()
model.add(Lambda(lambda x: x/255., input_shape=input_dim))
model.add(BatchNormalization())
model.add(Conv2D(24, (5,5), activation='relu', strides=(2, 2)))
model.add(Conv2D(36, (5,5), activation='relu', strides=(2, 2)))
model.add(Conv2D(48, (5,5), activation='relu', strides=(2, 2)))
model.add(Conv2D(64,(3,3), activation='relu', strides=(1, 1)))
model.add(Conv2D(64,(3,3), activation='relu', strides=(1, 1)))

model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(1164, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='softmax'))

adam = Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
lambda_4 (Lambda)            (None, 66, 200, 3)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 66, 200, 3)        12        
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 3, 20, 64)         27712     
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 1, 18, 64)         36928     
_________________________________________________________________
flatten_4 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1152)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 1164)              1342092   
_________________________________________________________________
dense_21 (Dense)             (None, 100)               116500    
_________________________________________________________________
dense_22 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_23 (Dense)             (None, 10)                510       
_________________________________________________________________
dense_24 (Dense)             (None, 10)                110       
=================================================================
Total params: 1,595,622
Trainable params: 1,595,616
Non-trainable params: 6
_________________________________________________________________

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_4 (Lambda)            (None, 66, 200, 3)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 66, 200, 3)        12        
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 3, 20, 64)         27712     
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 1, 18, 64)        

In [ ]:
Record ID	Shared On
(UNIX timestamp)	Share type	Share To User

In [1700]:
df = pd.read_clipboard('|')

In [1702]:
df.columns = ['a','Record ID','Shared On (UNIX timestamp)', 'Share type','Share To User', 'b']

In [1705]:
df[['Record ID','Shared On (UNIX timestamp)', 'Share type','Share To User']].to_dict()

{'Record ID': {0: 1, 1: 2, 2: 3, 3: 1, 4: 3, 5: 4, 6: 1, 7: 2, 8: 1},
 'Shared On (UNIX timestamp)': {0: 1611872850,
  1: 1611872851,
  2: 1611872852,
  3: 1611872853,
  4: 1611872854,
  5: 1611872855,
  6: 1611872856,
  7: 1611872857,
  8: 1611872858},
 'Share type': {0: ' shared        ',
  1: ' shared        ',
  2: ' shared        ',
  3: ' share_removed ',
  4: ' share_removed ',
  5: ' shared        ',
  6: ' shared        ',
  7: ' share_removed ',
  8: ' share_removed '},
 'Share To User': {0: ' user A        ',
  1: ' user B        ',
  2: ' user B        ',
  3: ' user A        ',
  4: ' user B        ',
  5: ' user C        ',
  6: ' user A        ',
  7: ' user B        ',
  8: ' user A        '}}

In [1775]:
df = pd.DataFrame({'Record ID': {0: 1, 1: 2, 2: 3, 3: 1, 4: 3, 5: 4, 6: 1, 7: 2, 8: 1},
 'Shared On (UNIX timestamp)': {0: 1611872850,
  1: 1611872851,
  2: 1611872852,
  3: 1611872853,
  4: 1611872854,
  5: 1611872855,
  6: 1611872856,
  7: 1611872857,
  8: 1611872858},
 'Share type': {0: ' shared        ',
  1: ' shared        ',
  2: ' shared        ',
  3: ' share_removed ',
  4: ' share_removed ',
  5: ' shared        ',
  6: ' shared        ',
  7: ' share_removed ',
  8: ' share_removed '},
 'Share To User': {0: ' user A        ',
  1: ' user B        ',
  2: ' user B        ',
  3: ' user A        ',
  4: ' user B        ',
  5: ' user C        ',
  6: ' user A        ',
  7: ' user B        ',
  8: ' user A        '}})

df['Share type'] = df['Share type'].str.strip()

In [1776]:
df

,Record ID,Shared On (UNIX timestamp),Share type,Share To User
0,1,1611872850,shared,user A
1,2,1611872851,shared,user B
2,3,1611872852,shared,user B
3,1,1611872853,share_removed,user A
4,3,1611872854,share_removed,user B
5,4,1611872855,shared,user C
6,1,1611872856,shared,user A
7,2,1611872857,share_removed,user B
8,1,1611872858,share_removed,user A


In [1777]:
#Check state of access as 1 or -1
df['state'] = df['Share type'].apply(lambda x: 1 if x=='shared' else -1)

#Groupby and filter for rows with state >=1
df[df.groupby(['Record ID', 'Share To User'])['state'].transform(sum)>=1].drop('state', axis=1)

,Record ID,Shared On (UNIX timestamp),Share type,Share To User
5,4,1611872855,shared,user C


In [ ]:
df.assign(age3=)

In [1755]:
df['Share To User'].unique()

array([' user A        ', ' user B        ', ' user C        '],
      dtype=object)

In [1774]:
cond = df.assign(state = lambda x: np.where(x['Share type']=='shared', 1, -1))\
         .groupby(['Record ID','Share To User'])['state']\
         .transform(sum)>=1

df[cond]

,Record ID,Shared On (UNIX timestamp),Share type,Share To User
5,4,1611872855,shared,user C


In [1730]:
df.assign(state=lambda x: 1 if  else -1)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [1789]:
print(pd.read_clipboard('|'))

   Unnamed: 0   Record ID    Shared On<br/>(UNIX timestamp)    \
0         NaN  ------------                      ------------   
1         NaN   1                                 1611872850    
2         NaN   2                                 1611872851    
3         NaN   3                                 1611872852    
4         NaN   1                                 1611872853    

    Share type       Share To User   Unnamed: 5  
0  ---------------  ---------------         NaN  
1   shared           user A                 NaN  
2   shared           user B                 NaN  
3   shared           user B                 NaN  
4   share_removed    user A                 NaN  


In [1813]:
a = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]])

b = np.array([2, 1, 2])

In [ ]:
np.array([[1, 2, 3],
          [4, 0, 6],
          [0, 0, 0]])

In [1831]:
from tensorflow.keras.layers import Input

In [1832]:
inp = Input(input_shape, name="input")
label_input = Input((1,), name="label_input")
aug_models = [get_augmentation_model() for i in range(n_classes)]
augmentation = aug_models[label_input](inp)
x = encoder_model(inp)
y = encoder_model(augmentation)
model = Model(inputs=[inp, label_input], outputs=[x, y])

NameError: name 'input_shape' is not defined

In [1842]:
a = np.array([[1,2,3],[4,5,6]])
b = np.array([1,7])

np.hstack([a,b[:,None]])

array([[1, 2, 3, 1],
       [4, 5, 6, 7]])

In [1853]:
a = [[1,2,'a','b'],[3,4,'c','d'],[5,6,'e','f']]

In [1855]:
import numpy as np

x,y = np.array(a)[:,0:2], np.array(a)[:,2:]
print(x)
print(y)

[['1' '2']
 ['3' '4']
 ['5' '6']]
[['a' 'b']
 ['c' 'd']
 ['e' 'f']]


In [1859]:
x,y = [i[:2] for i in a], [i[2:] for i in a]
print(x)
print(y)

[[1, 2], [3, 4], [5, 6]]
[['a', 'b'], ['c', 'd'], ['e', 'f']]

[[1, 2], [3, 4], [5, 6]]
[['a', 'b'], ['c', 'd'], ['e', 'f']]


In [1863]:
def list_squared(m, n):
    results = []
    for i in range (m, n):
        s = 0
        for j in range (1, i+1):
            if i % j == 0:
                s += j**2
        if int(s**(1/2))**2 == s:
            results.append([i, s])
    return results

In [1893]:
df1 = pd.DataFrame([[1,2],[3,4],[5,6],[7,8]], columns=['item','price'])
df2 = pd.DataFrame([[1,2],[3,4],[4,6],[6,8]], columns=['item','price'])
df3 = pd.DataFrame([[1,2],[4,4],[5,6],[6,8]], columns=['item','price'])

dfs = [df1,df2,df3]
dfs = [j.set_index('item').add_suffix('_'+str(i)) for i,j in enumerate(dfs)]
print(pd.concat(dfs, axis=1).reset_index())

   item  price_0  price_1  price_2
0     1      2.0      2.0      2.0
1     3      4.0      4.0      NaN
2     4      NaN      6.0      4.0
3     5      6.0      NaN      6.0
4     6      NaN      8.0      8.0
5     7      8.0      NaN      NaN


In [1882]:
df.to_numpy()[:,1]

array([1, 2, 3, 4, 5, 6, 7], dtype=object)

In [1887]:
df = pd.DataFrame({'group':['a','a','b','b','c','c','c'], 'score':[1,2,3,4,5,6,7]})

In [1888]:
df.groupby('group')['score'].transform('mean')

0    1.5
1    1.5
2    3.5
3    3.5
4    6.0
5    6.0
6    6.0
Name: score, dtype: float64

In [1890]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Lambda, BatchNormalization, Conv2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

input_dim = (160,305,3)

model = Sequential()
model.add(Lambda(lambda x: x/255., input_shape=input_dim))
model.add(BatchNormalization())
model.add(Conv2D(24, (5,5), activation='relu', strides=(2, 2)))
model.add(Conv2D(36, (5,5), activation='relu', strides=(2, 2)))
model.add(Conv2D(48, (5,5), activation='relu', strides=(2, 2)))
model.add(Conv2D(64,(3,3), activation='relu', strides=(1, 1)))
model.add(Conv2D(64,(3,3), activation='relu', strides=(1, 1)))

model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(1164, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

adam = Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss='mse')

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
lambda_6 (Lambda)            (None, 160, 305, 3)       0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 160, 305, 3)       12        
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 78, 151, 24)       1824      
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 37, 74, 36)        21636     
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 17, 35, 48)        43248     
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 15, 33, 64)        27712     
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 13, 31, 64)        36928     
_________________________________________________________________
flatten_6 (Flatten)          (None, 25792)             0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 25792)             0         
_________________________________________________________________
dense_30 (Dense)             (None, 1164)              30023052  
_________________________________________________________________
dense_31 (Dense)             (None, 100)               116500    
_________________________________________________________________
dense_32 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_33 (Dense)             (None, 10)                510       
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 11        
=================================================================
Total params: 30,276,483
Trainable params: 30,276,477
Non-trainable params: 6
_________________________________________________________________

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_6 (Lambda)            (None, 160, 305, 3)       0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 160, 305, 3)       12        
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 78, 151, 24)       1824      
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 37, 74, 36)        21636     
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 17, 35, 48)        43248     
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 15, 33, 64)        27712     
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 13, 31, 64)       

In [1900]:
import tensorflow as tf
import numpy as np

a = tf.constant(
    np.array([
        [ 8, 51,  1, 30,  3, 30],
        [ 1,  5,  2,  1,  1,  1],
        [11, 29,  1,  1,  1,  1],
        [ 1, 43,  1, 44, 27, 45],
        [ 1,  1,  1,  1,  1, 19],
        ])
)
l = tf.constant(np.array([[2], [1], [1], [2], [3]]))
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, activation='softmax', input_shape=[6])
])
optimizer = tf.keras.optimizers.Adam(lr=1e-8)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer)
print(model.summary())
history = model.fit(a,l, epochs=3, verbose=2)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 3)                 21        
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3


InvalidArgumentError:  Received a label value of 3 which is outside the valid range of [0, 3).  Label values: 3 2 1 2 1
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-1900-03a877e88a34>:20) ]] [Op:__inference_train_function_4806]

Function call stack:
train_function


In [1948]:
def list_squared(m, n):
    results = []
    for i in range(m, n):
        sum = 0
        for j in range (1, i+1):
            #print(i,j)
            if i % j == 0:
                sum += j**2
    print(sum)
#         if int(sum**(1/2))**2 == sum: #checking if number is a perfect square number
#             results.append([i, sum])
    return results

In [1949]:
list_squared(1, 15)

250


[]

In [1905]:
list_squared(42, 250)

[[42, 2500], [246, 84100]]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [1919]:

denominator

array([ 1,  2,  2,  3,  3,  3,  4,  4,  4,  4,  5,  5,  5,  5,  5,  6,  6,
        6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,
        8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
       14, 14, 14])

In [ ]:
numerator = np.

In [1922]:
np.unique(denominator, return_index=True, return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([ 0,  1,  3,  6, 10, 15, 21, 28, 36, 45, 55, 66, 78, 91]),
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]))

array([ 2,  3,  2,  4,  3,  3,  5,  4,  4,  4,  6,  5,  5,  5,  5,  7,  6,
        6,  6,  6,  6,  8,  7,  7,  7,  7,  7,  7,  9,  8,  8,  8,  8,  8,
        8,  8, 10,  9,  9,  9,  9,  9,  9,  9,  9, 11, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 12, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 13, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 14, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 15, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,
       14, 14, 14])

[1, 1, 2, 1, 2, 3, 1, 2, 3, 4]

In [1941]:
denominator = np.repeat(np.arange(1,15), np.arange(1,15))
numerator = [j for i in range(1,15) for j in range(1,i+1)]



In [1950]:
np.sum(denominator[np.mod(numerator,denominator)==0]**2)

1015

In [1951]:
np.mod(numerator,denominator)

array([ 0,  1,  0,  1,  2,  0,  1,  2,  3,  0,  1,  2,  3,  4,  0,  1,  2,
        3,  4,  5,  0,  1,  2,  3,  4,  5,  6,  0,  1,  2,  3,  4,  5,  6,
        7,  0,  1,  2,  3,  4,  5,  6,  7,  8,  0,  1,  2,  3,  4,  5,  6,
        7,  8,  9,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,
        3,  4,  5,  6,  7,  8,  9, 10, 11,  0,  1,  2,  3,  4,  5,  6,  7,
        8,  9, 10, 11, 12,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,
       12, 13,  0])

In [1952]:
np.tril_indices(5)[1] + 1

array([1, 1, 2, 1, 2, 3, 1, 2, 3, 4, 1, 2, 3, 4, 5])